# Aufgabe 2 - Tiefe Einblicke ins Institut

Den Code immer nachvollziehbar kommentieren! Bitte beachtet, dass das Notebook von Anfang bis Ende ohne Fehler durchlaufen muss und dass die requirements.txt Datei aktualisiert wird.

## Teilaufgabe a): Trainings-und Testdatenset
## Teilaufgabe b): Verteilung RGB

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from PIL import Image


def thumbnail(img):
    img_list = np.asarray(img.resize((30, 30))).reshape(1, 2700)
    final_list = np.array([item for sublist in img_list for item in sublist])
    return final_list


class PicData:
    def __init__(self, name):
        file_paths = {
            'prof': "Professorenbuero",
            'hall': "Flur",
            'lab': "Labor",
            'tea': "Teekueche"
        }
        name_to_label = {
            'prof': 1,
            'hall': 2,
            'lab': 3,
            'tea': 4
        }
        pic_amount = len(os.listdir(f"Bilder/{file_paths[name]}"))
        self.path = file_paths[name]
        self.pics = [Image.open(f"Bilder/{self.path}/{self.path + str(i + 1)}.jpg") for i in range(pic_amount)]
        self.data = np.array([img.histogram() for img in self.pics])
        random.shuffle(self.data)
        #self.labels = np.array([name_to_label[name] for _ in range(len(self.data))])
        #self.img_train, self.img_test, self.lbl_train, self.lbl_test = train_test_split(self.data, self.labels, test_size=0.2)
        self.train = [(self.data[i], name_to_label[name]) for i in range(int(0.8 * len(self.data)))]
        self.test  = [(self.data[i], name_to_label[name]) for i in range(int(0.8 * len(self.data)), len(self.data))]

prof_pics = PicData('prof')
hall_pics = PicData('hall')
lab_pics = PicData('lab')
tea_pics = PicData('tea')

'''print(len(prof_pics.train))
print(len(prof_pics.test))
print(prof_pics.train[0])
print(prof_pics.img_test)
print(prof_pics.lbl_train)
print(prof_pics.lbl_test)
'''
"""
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0, 0].hist(prof_pics.histogram, label='prof', bins=768)
ax[0, 1].hist(hall_pics.histogram, label='hall', bins=768)
ax[1, 0].hist(lab_pics.histogram, label='lab', bins=768)
ax[1, 1].hist(tea_pics.histogram, label='tea', bins=768)

ax[0, 0].legend()
ax[0, 1].legend()
ax[1, 0].legend()
ax[1, 1].legend()

plt.show()
"""




20
5
(array([144, 158, 188, 343, 395, 496, 438, 442, 434, 411, 400, 325, 313,
       254, 252, 217, 175, 174, 144, 108, 127, 141, 149, 145, 146, 161,
       159, 168, 142, 159, 196, 194, 238, 258, 226, 200, 218, 217, 221,
       203, 212, 226, 242, 286, 258, 270, 271, 254, 250, 228, 230, 220,
       135, 153, 142, 126, 142, 130, 142, 151, 161, 145, 160, 193, 174,
       184, 195, 216, 229, 192, 239, 206, 232, 205, 186, 225, 189, 215,
       180, 214, 207, 229, 191, 242, 276, 321, 339, 351, 352, 271, 286,
       258, 263, 272, 222, 246, 213, 226, 251, 240, 231, 237, 219, 216,
       198, 205, 206, 195, 215, 231, 259, 298, 296, 302, 364, 413, 474,
       468, 527, 553, 467, 411, 387, 358, 328, 315, 313, 282, 312, 292,
       313, 400, 360, 367, 359, 291, 290, 342, 332, 343, 357, 344, 291,
       297, 259, 262, 244, 250, 248, 237, 274, 259, 276, 277, 233, 230,
       214, 243, 239, 215, 228, 265, 243, 226, 193, 188, 215, 206, 210,
       195, 203, 190, 198, 182, 190, 163, 190, 184, 195, 1

"\nfig, ax = plt.subplots(2, 2, figsize=(10, 10))\nax[0, 0].hist(prof_pics.histogram, label='prof', bins=768)\nax[0, 1].hist(hall_pics.histogram, label='hall', bins=768)\nax[1, 0].hist(lab_pics.histogram, label='lab', bins=768)\nax[1, 1].hist(tea_pics.histogram, label='tea', bins=768)\n\nax[0, 0].legend()\nax[0, 1].legend()\nax[1, 0].legend()\nax[1, 1].legend()\n\nplt.show()\n"

## Teilaufgabe c): Training und Test mit Random Forest

In [44]:
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier

complete_train = list(prof_pics.train) + list(hall_pics.train) + list(lab_pics.train) + list(tea_pics.train)
complete_test = list(prof_pics.test) + list(hall_pics.test) + list(lab_pics.test) + list(tea_pics.test)
random.shuffle(complete_train)
random.shuffle(complete_test)


X_train = [item[0] for item in complete_train]
y_train = [item[1] for item in complete_train]
X_test = [item[0] for item in complete_test]
y_test = [item[1] for item in complete_test]

rf = RandomForestClassifier(n_estimators=300)
rf.fit(X_train, y_train)

y_pred_train = rf.predict(X_train)
print(r2_score(y_train, y_pred_train))

'''complete_image_list = list(prof_pics.img_train) + list(hall_pics.img_train) + list(lab_pics.img_train) + list(tea_pics.img_train)
complete_label_list = list(prof_pics.lbl_train) + list(hall_pics.lbl_train) + list(lab_pics.lbl_train) + list(tea_pics.lbl_train)
rf.fit(complete_image_list, complete_label_list)'''


1.0


'complete_image_list = list(prof_pics.img_train) + list(hall_pics.img_train) + list(lab_pics.img_train) + list(tea_pics.img_train)\ncomplete_label_list = list(prof_pics.lbl_train) + list(hall_pics.lbl_train) + list(lab_pics.lbl_train) + list(tea_pics.lbl_train)\nrf.fit(complete_image_list, complete_label_list)'

In [45]:
current_test_list = complete_test
current_test_data = [item[0] for item in current_test_list]
current_test_lbl = [item[1] for item in current_test_list]

y_pred = rf.predict(current_test_data)
print(y_pred)
print(current_test_lbl)

print(r2_score(current_test_lbl, y_pred))


errors = abs(y_pred - current_test_lbl)
print('Mean Absolute Error:', str(round(np.mean(errors), 2)) + '°.')

print("Score:", rf.score(current_test_data, current_test_lbl))

mape = 100 * (errors / current_test_lbl)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

[2 3 1 1 1 4 2 4 1 1 4 3 2 3 2 3 4 3 3 2]
[2, 3, 4, 1, 1, 4, 1, 2, 1, 1, 4, 4, 2, 3, 2, 3, 4, 3, 3, 2]
0.4
Mean Absolute Error: 0.35°.
Score: 0.8
Accuracy: 85.0 %.


## Teilaufgabe d): Hyperparameteroptimierung

## Teilaufgabe e): Dimensionalitätsreduktion